In [1]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sklearn.decomposition import FastICA
import requests
from tqdm import tqdm
from more_itertools import locate

def import_data():
    sb2_fits = fits.open("/Users/blaine/Documents/astrodata/apogee_sb2s-v1_0.fits", memmap=True)
    a = sb2_fits[1].data
    all_visit = fits.open("/Users/blaine/Documents/astrodata/allVisit-dr17-synspec_rev1.fits", memmap=True)
    b = all_visit[1].data
    all_star = fits.open("/Users/blaine/Documents/astrodata/allStar-dr17-synspec_rev1.fits", memmap=True)
    c = all_star[1].data

    return a,b,c

def filter_sb2(sb2):
    ret = []
    for i in range(0, sb2.shape[0]):
        arr = sb2[i][10]
        if arr[0] >= 3 and arr[1] >= 3 and arr[2] == 0 and arr[3] == 0:
            ret.append(i)
    return ret

def filter_allVisit(visit_id, visit, id_lst):
    ret = []
    for dat in tqdm(id_lst):
        dat_mjd = dat[0]
        dat_id = dat[1]
        temp = list(locate(visit_id, lambda x:x[1] == dat_mjd and x[0] == dat_id))
        if len(temp) > 0:
            ret.append(visit[temp[0]][0])
        else:
            continue
    return ret

def dict_display(dict,N):
    for i in range(0,N):
        print(list(dict.items())[i])
        print("")

In [10]:
visit_id_list

[['VESTA', 56398],
 ['VESTA', 56778],
 ['2M00000002+7417074', 56936],
 ['2M00000002+7417074', 56971],
 ['2M00000002+7417074', 56933],
 ['2M00000019-1924498', 58037],
 ['2M00000019-1924498', 57293],
 ['2M00000019-1924498', 58384],
 ['2M00000032+5737103', 56261],
 ['2M00000032+5737103', 56284],
 ['2M00000032+5737103', 56257],
 ['2M00000032+5737103', 56651],
 ['2M00000032+5737103', 55874],
 ['2M00000032+5737103', 56643],
 ['2M00000035-7323394', 59155],
 ['2M00000035-7323394', 59158],
 ['2M00000035-7323394', 59157],
 ['2M00000035-7323394', 59162],
 ['2M00000035-7323394', 59156],
 ['2M00000035-7323394', 59147],
 ['2M00000035-7323394', 59166],
 ['2M00000035-7323394', 59165],
 ['2M00000035-7323394', 59160],
 ['2M00000035-7323394', 59154],
 ['2M00000035-7323394', 59159],
 ['2M00000035-7323394', 59164],
 ['2M00000068+5710233', 55874],
 ['2M00000068+5710233', 56643],
 ['2M00000068+5710233', 56651],
 ['2M00000103+1525513', 59077],
 ['2M00000103+1525513', 59074],
 ['2M00000133+5721163', 58766],
 [

In [2]:
pip install numpy==1.21.5

  Using cached numpy-1.21.5-cp39-cp39-macosx_10_9_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.1.1 which is incompatible.
spleeter 2.3.0 requires numpy<1.20.0,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
sb2_raw, visit_raw, star_raw = import_data()
#Filter sb2 data to have both ratings above 3
sb2_idx = filter_sb2(sb2_raw)

In [3]:
#Condense SB2 Data into a list of all filtered elements
sb2_data = [sb2_raw[idx] for idx in sb2_idx]

In [6]:
#Create a Dictionary of all elements of sb2 logged by appogee ID
sb2_mjd = [[int(np.floor(element[1])), element] for element in sb2_data]

In [5]:
#Convert fits format to list of data
visit_id_list = [[visit[0], visit[7]] for visit in visit_raw]

In [7]:
sb2_info_list = [[b2[0],b2[1][0]] for b2 in sb2_mjd] 

In [8]:
#Get the index of all_visit data that matches sb2_appid_lst
filtered_allVisit = filter_allVisit(visit_id_list, visit_raw, sb2_info_list)

  0%|                                     | 22/16690 [01:18<16:26:49,  3.55s/it]


KeyboardInterrupt: 

In [107]:
len(filtered_allVisit)

34695

In [108]:
len(sb2_mjd)

16690

In [25]:
allvisit_mjd_list = [x[7] for x in filtered_allVisit]
unique_all_visit = list(dict.fromkeys(allvisit_mjd_list))

In [91]:
a_visit_file_name = []
un = []
cnt = 0
for mjd in tqdm(sb2_mjd):
    try:
        idx = allvisit_mjd_list.index(mjd[0])
        a_visit_file_name.append([filtered_allVisit[idx][0],mjd[0],filtered_allVisit[idx][3]])
    except ValueError:
        cnt += 1
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16690/16690 [02:41<00:00, 103.43it/s]


In [93]:
sb2_mjd[7]

[56666,
 ('2M00005196+0022164', 56666.57177999988, 0.216528, 0.371225, array([0.25662703, 0.21850794,        nan,        nan]), array([ 29.46407411, -17.65743485,          nan,          nan]), array([52.53603337, 47.88960938,         nan,         nan]), array([0.05002191, 0.06486619,        nan,        nan]), array([ 9.80478907, 10.19373827,         nan,         nan]), array([13.88279918, 13.73243388,         nan,         nan]), array([4, 3, 0, 0]), 2)]

In [99]:
sb2_mjd[39]

[56666,
 ('2M00024876-0018006', 56666.57177999988, 0.703181, -0.300184, array([0.40185354, 0.383164  ,        nan,        nan]), array([-24.4777411 ,  10.32298773,          nan,          nan]), array([26.81078694, 28.1441704 ,         nan,         nan]), array([0.03413323, 0.03321208,        nan,        nan]), array([1.80823461, 1.94489359,        nan,        nan]), array([4.03958774, 4.42918817,        nan,        nan]), array([4, 4, 0, 0]), 2)]

In [92]:
a_visit_file_name

[['2M00004521-7219055', 59154, 'asVisit-dr17-12776-59154-069.fits'],
 ['2M00004521-7219055', 59155, 'asVisit-dr17-12776-59155-069.fits'],
 ['2M00004521-7219055', 59156, 'asVisit-dr17-12776-59156-069.fits'],
 ['2M00004521-7219055', 59157, 'asVisit-dr17-12776-59157-069.fits'],
 ['2M00004521-7219055', 59164, 'asVisit-dr17-12776-59164-070.fits'],
 ['2M00004521-7219055', 59165, 'asVisit-dr17-12776-59165-070.fits'],
 ['2M00004521-7219055', 59166, 'asVisit-dr17-12776-59166-070.fits'],
 ['2M00005196+0022164', 56666, 'apVisit-dr17-7063-56666-028.fits'],
 ['2M00005196+0022164', 57677, 'apVisit-dr17-9243-57677-022.fits'],
 ['2M00005196+0022164', 57654, 'apVisit-dr17-9243-57654-028.fits'],
 ['2M00015725+5726512', 58776, 'apVisit-dr17-12184-58776-247.fits'],
 ['2M00015725+5726512', 58861, 'apVisit-dr17-12184-58861-253.fits'],
 ['2M00023036+8524194', 57822, 'apVisit-dr17-9069-57822-224.fits'],
 ['2M00023036+8524194', 58150, 'apVisit-dr17-9069-58150-224.fits'],
 ['2M00023036+8524194', 59059, 'apVisit

In [87]:
temp_1

'asVisit-dr17-12776-59154-069.fits'

In [83]:
start = 0
for i in tqdm(range(0,len(temp_1))):
    try:
        mjd_0 = all_sb2_mjd[i]
        idx = all_visit_mjd.index(mjd_0)
        extract_data_0 = binary[idx]
        extract_data_1 = filtered_sb2_data[i]
        
        filename = str(extract_data_0[3])
        mjd = int(extract_data_0[7])
        apogeeID = extract_data_1[0]
        visitno = int(extract_data_0[4])
        try:
            testing_data = fits.open("../../../data/" + extract_data_0[3], memmap=True)
        except FileNotFoundError:
            continue
        fluxa = [float(a) for a in testing_data[1].data[0]]
        wavelengtha = [float(a) for a in testing_data[4].data[0]]
        fluxb = [float(a) for a in testing_data[1].data[1]]
        wavelengthb = [float(a) for a in testing_data[4].data[1]]
        fluxc = [float(a) for a in testing_data[1].data[2]]
        wavelengthc = [float(a) for a in testing_data[4].data[2]]
        ampa = float(extract_data_1[4][0])
        vhelioa = float(extract_data_1[5][0])
        fwhma = float(extract_data_1[6][0])
        ampb = float(extract_data_1[4][1])
        vheliob = float(extract_data_1[5][1])
        fwhmb = float(extract_data_1[6][1])
        SNR = float(extract_data_0["SNR"])
        if np.isnan(ampa) or np.isnan(SNR) or np.isnan(vhelioa) or np.isnan(ampa) or np.isnan(fwhma) or np.isnan(ampb) or np.isnan(vheliob) or np.isnan(fwhmb):
            continue
        attr = ["filename", "apogeeID", "visitno", "fluxa", "wavelengtha", "fluxb", "wavelengthb", "fluxc", "wavelengthc",
                     "ampa", "vhelioa", "fwhma", "ampb", "vheliob", "fwhmb", "SNR", "mjd"]
        vals = [filename, apogeeID, visitno, fluxa, wavelengtha, fluxb, wavelengthb, fluxc, wavelengthc,
                         ampa, vhelioa, fwhma, ampb, vheliob, fwhmb, SNR, mjd]
        to_json = {a:b for a,b in zip(attr,vals)}
        requests.post('http://127.0.0.1:5000/test', json=to_json)
    except ValueError:
        print("bruh")
        pass

['asVisit-dr17-12776-59154-069.fits',
 'asVisit-dr17-12776-59155-069.fits',
 'asVisit-dr17-12776-59156-069.fits',
 'asVisit-dr17-12776-59157-069.fits',
 'asVisit-dr17-12776-59164-070.fits']